# Building Fast Queries on a CSV 
The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

- Given a laptop id, find the corresponding data.
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.


We create a `class` named `Inventory`.
The class will have a `constructor` that takes the `csv_filename as an argument`. The encoding fo the file provided in our data directory is `UTF-8`.

Before we do that lets just explore the file:

In [1]:
import csv
with open('data/laptops.csv') as f:
    reader=csv.reader(f)
    rows=list(reader)
rows[:5]


[['Id',
  'Company',
  'Product',
  'TypeName',
  'Inches',
  'ScreenResolution',
  'Cpu',
  'Ram',
  'Memory',
  'Gpu',
  'OpSys',
  'Weight',
  'Price'],
 ['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898'],
 ['3362737',
  'HP',
  '250 G6',
  'Notebook',
  '15.6',
  'Full HD 1920x1080',
  'Intel Core i5 7200U 2.5GHz',
  '8GB',
  '256GB SSD',
  'Intel HD Graphics 620',
  'No OS',
  '1.86kg',
  '575'],
 ['9722156',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '15.4',
  'IPS Panel Retina Display 2880x1800',
  'Intel Core i7 2.7GHz',
  '16GB',
  '512GB SSD',
  'AMD Radeon Pro 455',
  'macOS',
  '1.83kg',
  '2537']]

## Inventory class

In [2]:

class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}',encoding='utf8') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])

### Instance creation for testing and returning the header (column names)

In [3]:
my_invetory=Inventory('laptops.csv')
my_invetory.header

['Id',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price']

### Checking the number of rows

In [4]:
len(my_invetory.rows)

1303

## Finding a Laptop From the Id 
### Adding `get_laptop_from_id()` method to the inventory class
The time complexity fir this lookup is `O(R)` where R is the number of rows

In [5]:
class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if row[0]==laptop_id:
                return row
         
        return None
            

### Testing the method 

In [6]:
my_invetory=Inventory('laptops.csv')
my_invetory.get_laptop_from_id('3362737')


['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [7]:
print(my_invetory.get_laptop_from_id('3362736'))

None


## Improving Id Lookups
### Implementing a second faster method
The `get_laptop_from_id()` method has time complexity O(R) where R is the number of rows.To improve the time complexity of finding a laptop with a given id we precompute a dictionary that maps laptop ids to rows. Dictoniary lookups are perfomed in `O(1)`
The trade-off is using more memory to store the dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Below we define `get_laptop_from_id_fast` using a disctionary lookup note the dictionary will be defined in the constractor as following:

- the laptop ID is the key 
- the value will be the row corresponding to that laptop ID


In [8]:
class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])
            
        #defining the dictionary for fast look-up 
        self.id_to_row={}
        for row in self.rows:
            self.id_to_row[row[0]]=row
    
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if row[0]==laptop_id:
                return row
        return None
    
    # new faster function is defined using dict                    
    def get_laptop_from_id_fast(self,laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]      
        return None

### Testing the method 

In [9]:
my_invetory=Inventory('laptops.csv')
my_invetory.get_laptop_from_id_fast('3362737')

['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [10]:
print(my_invetory.get_laptop_from_id('3362736'))

None


# Performance comparision:
To compare the performance of the two methods. The idea is to generate random IDs using the `random` module. Then, use both methods to lookup these same IDs. We will use the `time` module to measure the execution time of each lookup and, for each method, add all times together. We start by importing the needed modules:

In [11]:
import time
import random 

We generate a list named `ids` with 10,000 random values between "1000000" and "9999999" (this is the id range). Note the use of strings rather than integers. This is because the IDs in the CSV files are read a strings, not integers. We will convert the integers to strings using the str() function.

In [12]:
ids= [str(random.randint(1000000,9999999)) for _ in range(10000)]

In [13]:
print(len(ids))
type(ids[1])

10000


str

Next we create an instance of `Inventory` by giving 'laptops.csv' as argument and initialize a variable named `total_time_no_dict` and set it to 0. This variable will aggregate the times of calling get_laptop_from_id().

In [14]:
test_invetory=Inventory('laptops.csv')
total_time_no_dict=0

for id in ids:
    start=time.time()
    test_invetory.get_laptop_from_id(id)
    end=time.time()
    total_time_no_dict+=(end-start)

total_time_no_dict    

0.9290003776550293

next we initialize another variable named `total_time_dict` and set it to 0. This variable will aggregate the times of calling `get_laptop_from_id_fast()`.

In [15]:
total_time_dict=0

for id in ids:
    start=time.time()
    test_invetory.get_laptop_from_id_fast(id)
    end=time.time()
    total_time_dict+=(end-start)

total_time_dict 

0.0110015869140625

# Analysing the results


In [16]:
print(f'We can see a significant improvment in performance. The new method is:\
{total_time_no_dict/total_time_dict} times faster.')

We can see a significant improvment in performance. The new method is:84.44239771151179 times faster.


# Laptop promotion:
Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory:


<center><img src="https://dq-content.s3.amazonaws.com/481/laptop_promotion.png"/></center>       


laptop promotion
The prices of these laptops are \\$1,339, \\$898, and \\$575. Say we offered a gift card of $\\2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is \\$2,237 (\\$1,339 plus \\$898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full \\$2,500.

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

We write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [17]:
class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])
            
        #defining the dictionary for fast look-up 
        self.id_to_row={}
        for row in self.rows:
            self.id_to_row[row[0]]=row
    
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if row[0]==laptop_id:
                return row
        return None
    
    # new faster function is defined using dict                    
    def get_laptop_from_id_fast(self,laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]      
        return None
    
    def check_promotion_dollars(self,dollars):
        for row in self.rows:
            if row[-1]==dollars:
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row[-1]+row2[-1]== dollars:
                    return True
        return False

### Testing the new method:

In [18]:
inventory = Inventory('laptops.csv')               
print(inventory.check_promotion_dollars(1000))     
print(inventory.check_promotion_dollars(442))  

True
False


## Optimizing Laptop Promotion
Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory `self.inventory`. Then we can check in constant time whether there is a laptop with a given price.

We are going to use the set to implement `check_promotion_dollars_fast()`

In [19]:
class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])
            
        # using a set to store all prices
        self.prices=set()
        for row in self.rows:
            self.prices.add(row[-1])
        
            
        #defining the dictionary for fast look-up 
        self.id_to_row={}
        for row in self.rows:
            self.id_to_row[row[0]]=row
    
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if row[0]==laptop_id:
                return row
        return None
    
    # new faster function is defined using dict                    
    def get_laptop_from_id_fast(self,laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]      
        return None
    
    def check_promotion_dollars(self,dollars):
        for row in self.rows:
            if row[-1]==dollars:
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row[-1]+row2[-1]== dollars:
                    return True
        return False
    
    
    # new check_promotion_dollars_fast() using the set we created for prices
    def check_promotion_dollars_fast(self,dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars-price in self.prices:
                return True
        return False           
            

### Testing check_promotion_dollars_fast()

In [20]:
inventory = Inventory('laptops.csv')                 
print(inventory.check_promotion_dollars_fast(1000))  
print(inventory.check_promotion_dollars_fast(442))  

True
False


### Comparing Promotion Functions
To do the time comparison simlart to before we are going to:

- Generate a list named `prices` with 100 random values between 100 and 5,000.
- Initialize a variable named `total_time_no_set` , `total_time_set` to aggregate the times of calling the old vs the new function
- compare the results

In [21]:
prices=[random.randint(100,5000) for _ in range(100)]
print(prices[-1])
len(prices)

4352


100

In [22]:
inventory = Inventory('laptops.csv')

total_time_no_set=0
for price in prices:
    start=time.time()
    inventory.check_promotion_dollars(price)
    end=time.time()
    total_time_no_set+=(end-start)

total_time_set=0
for price in prices:
    start=time.time()
    inventory.check_promotion_dollars_fast(price)
    end=time.time()
    total_time_set+=(end-start)
    
print(f'Thetotal time running the old promotion function is{total_time_no_set}.\
      \nThe total time using the new faster function is {total_time_set}.\
      \nWhich is {total_time_no_set/total_time_set} faster.')   
    

Thetotal time running the old promotion function is1.8959977626800537.      
The total time using the new faster function is 0.0009996891021728516.      
Which is 1896.5874075840686 faster.


# Finding laptops within a budget
we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars:


![](https://dq-content.s3.amazonaws.com/481/laptop_budget.png)

- At the end of the `__init__()` method, we will use the sorted() function to sort the rows by price and assign the result to `self.rows_by_price`
- Implement a method named `find_first_laptop_more_expensive()` that is based on the `binary search algorithm`. This method should take two arguments: self and price. It should return the index of the first row in self.rows_by_price whose price is higher than price. Return -1 if there is no such index.

In [23]:
class Inventory():
    def __init__(self,file_name):
        with open(f'data/{file_name}') as f:
            reader=csv.reader(f)
            rows=list(reader)
        self.header=rows[0]
        self.rows=rows[1:]
        for row in self.rows:
            row[-1]=int(row[-1])
            
        # using a set to store all prices
        self.prices=set()
        for row in self.rows:
            self.prices.add(row[-1])
        
        #sorting rows by prices
        self.rows_by_price=sorted(self.rows, key=lambda row: row[-1])
        
            
        #defining the dictionary for fast look-up 
        self.id_to_row={}
        for row in self.rows:
            self.id_to_row[row[0]]=row
    
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if row[0]==laptop_id:
                return row
        return None
    
    # new faster function is defined using dict                    
    def get_laptop_from_id_fast(self,laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]      
        return None
    
    def check_promotion_dollars(self,dollars):
        for row in self.rows:
            if row[-1]==dollars:
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row[-1]+row2[-1]== dollars:
                    return True
        return False
        
    def check_promotion_dollars_fast(self,dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars-price in self.prices:
                return True
        return False   
    
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start=0
        range_end=len(self.rows_by_price)-1
        while range_start<range_end:
            range_middle= (range_start+range_end)//2
            price=self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

# Testing the new method:

In [24]:
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


Checking the prices for indexes around 683 to see if we gor the correct index:

In [25]:
for index in range(680,685):
    print(f'price for index#:{index} is {inventory.rows_by_price [index][-1]}')

price for index#:680 is 999
price for index#:681 is 999
price for index#:682 is 1000
price for index#:683 is 1002
price for index#:684 is 1008
